In [1]:
%matplotlib notebook

from modsim import *

m = UNITS.meter
s = UNITS.second
kg = UNITS.kilogram
degree = UNITS.degree
radian = UNITS.radian

In [19]:
condition = Condition(x = 0 * m, 
                      y = 1 * m,
                      g = 9.8 * m/s**2,
                      mass = 145e-3 * kg,
                      diameter = 73e-3 * m,
                      rho = 1.2 * kg/m**3,
                      C_d = 0.3,
                      angle = 45 * degree,
                      velocity = 40 * m / s,
                      duration = 5.1 * s)

def make_system(condition):
    """Make a system object.
    
    condition: Condition object with angle, velocity, x, y,
               diameter, duration, g, mass, rho, and C_d
               
    returns: System object
    """
    unpack(condition)
    
    # convert angle to degrees
    theta = np.deg2rad(angle)
    
    # compute x and y components of velocity
    vx, vy = pol2cart(theta, velocity)
    
    # make the initial state
    init = State(x=x, y=y, vx=vx, vy=vy)
    
    # compute area from diameter
    area = np.pi * (diameter/2)**2
    
    # compute timestamps
    ts = linspace(0, duration, 101)
    
    return System(init=init, g=g, mass=mass, 
                  area=area, rho=rho, C_d=C_d, ts=ts)

def slope_func(state, t, system):
    """Computes derivatives of the state variables.
    
    state: State (x, y, x velocity, y velocity)
    t: time
    system: System object with g, rho, C_d, area, mass
    
    returns: sequence (vx, vy, ax, ay)
    """
    x, y, vx, vy = state
    unpack(system)
    
    a_grav = Vector(0, -g)

    v = Vector(vx, vy)
    
    f_drag = -rho * v.mag * v * C_d * area / 2
    a_drag = f_drag / mass
    
    a = a_grav + a_drag
    
    return vx, vy, a.x, a.y


def height_func(angle, condition):
    
    condition.set(angle=angle)
    system = make_system(condition)
    run_odeint(system, slope_func)
    
    T = interp_inverse(system.results.x)
    time_reached = T(94.5)
    
    Y = interpolate(system.results.y)
    height = Y(time_reached)
    
    return height

#res = max_bounded(height_func, [0, 90], condition)

def best_height(velocity, condition):
    condition.set(velocity=velocity)
    res = max_bounded(height_func, [0, 90], condition)
    
    angle = res.x
    
    height = height_func(angle, condition)
    return height

def error_func(velocity, condition):
    height = best_height(velocity, condition)
    
    error = height-11
    return error

In [22]:
system = make_system(condition)
run_odeint(system, slope_func)
res = max_bounded(height_func, [0, 90], condition)